# Process image folder

In [ ]:
import os
import cv2
from deepface import DeepFace

folder = './faces'

print(folder)

for file_name in os.listdir(folder):
    # Asume imágenes en formato png o jpg
    if file_name.endswith('.png') or file_name.endswith('.jpg'):
        # Procesa la imagen que asume hay cara, no fuerza la detección
        obj = DeepFace.analyze(img_path = os.path.join(folder, file_name), enforce_detection=False, actions =['age', 'gender', 'race', 'emotion'])
        print(file_name)
        print(obj)
        #print(obj["region"])
        #print(obj["age"])      
        #print(obj["gender"])      
        #print(obj["race"])       
        #print(obj["dominant_race"]) 
        #print(obj["emotion"])
        #print(obj["dominant_emotion"])
       
        


# Process from webcam

In [2]:
import cv2
import numpy as np
from deepface import DeepFace
#from playsound import playsound

vid = cv2.VideoCapture(0)

# font 
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 0) 
thickness = 2 
pos = (50, 50)

def blend(input_rgb, alpha, rgb, coord: (int, int)):
    cut = input_rgb[coord[1]:coord[1]+rgb.shape[0], coord[0]:coord[0]+rgb.shape[1]].astype(float)

    # Multiply the foreground with the alpha matte
    rgb = cv2.multiply(alpha, rgb)
    input_rgb = cv2.multiply(1.0 - alpha, cut)

    # Add the masked foreground and background
    input_rgb[coord[1]:coord[1]+rgb.shape[0], coord[0]:coord[0]+rgb.shape[1]] = cv2.add(rgb, cut).astype("uint8")
    return input_rgb

def read_transparent_png(filename):
    image_4channel = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    alpha_channel = image_4channel[:,:,3].astype(float) / 255
    rgb_channels = image_4channel[:,:,:3].astype(float)
    return (alpha_channel, rgb_channels)

balloons = read_transparent_png("assets/balloons.png")
confetti = read_transparent_png("assets/confetti.png")
rain = read_transparent_png("assets/rain.png")
sangre1 = read_transparent_png("assets/sangre1.png")
serious = read_transparent_png("assets/serious.png")
vignette = read_transparent_png("assets/vignette.png")

while(True):
    # fotograma a fotograma
    ret, frame = vid.read()
  
    if ret:  
        obj = DeepFace.analyze(img_path = frame, enforce_detection=False, actions =['gender', 'emotion'])
        print(obj)

        # Primera cara
        image = cv2.putText(frame, str(obj[0]['dominant_gender']+' '+obj[0]['dominant_emotion']), pos, font,  
                        fontScale, color, thickness, cv2.LINE_AA) 
        
        match obj[0]['dominant_emotion']: 
            case 'happy':

                imghsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype("float32")

                (h, s, v) = cv2.split(imghsv)
                s = s*2
                s = np.clip(s,0,255)
                imghsv = cv2.merge([h,s,v])

                frame = cv2.cvtColor(imghsv.astype("uint8"), cv2.COLOR_HSV2BGR)
            
            case 'sad':
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
            case 'angry':

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                image = np.zeros((image.shape), dtype = np.uint8)
                image[:,:,2] = frame
                frame = image

            case 'fear':
                pass

            case 'disgust':
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                image = np.zeros((image.shape), dtype = np.uint8)
                image[:,:,1] = frame
                frame = image

            case 'surprise':
                pass

            case 'neutral':
                # playsound('./assets/sad_sound.mp3')

                pass
                
            case _:
                pass

        # Muestra fotograma
        cv2.imshow('Vid', frame)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

2023-12-06 20:11:13.228664: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-06 20:11:13.229719: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-12-06 20:11:14.131850: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-12-06 20:11:14.583042: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10

[{'gender': {'Woman': 39.72950875759125, 'Man': 60.27049422264099}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 640, 'h': 480}, 'emotion': {'angry': 73.7567098430741, 'disgust': 7.194200998848543e-06, 'fear': 0.016735292780569313, 'happy': 1.3111264744548232, 'sad': 5.292158315520488, 'surprise': 0.37923271437940426, 'neutral': 19.24402633380576}, 'dominant_emotion': 'angry'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  2.54it/s]


[{'gender': {'Woman': 39.80438411235809, 'Man': 60.19561290740967}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 640, 'h': 480}, 'emotion': {'angry': 72.99826419332793, 'disgust': 7.168201110107889e-06, 'fear': 0.017881290790443514, 'happy': 1.3487088239143876, 'sad': 5.532868121012226, 'surprise': 0.3279509879441247, 'neutral': 19.774318608560396}, 'dominant_emotion': 'angry'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  2.29it/s]


...

[{'gender': {'Woman': 2.3842019960284233, 'Man': 97.61580228805542}, 'dominant_gender': 'Man', 'region': {'x': 256, 'y': 178, 'w': 148, 'h': 148}, 'emotion': {'angry': 46.684807538986206, 'disgust': 0.18667449476197362, 'fear': 3.425182029604912, 'happy': 0.9104493074119091, 'sad': 44.36790943145752, 'surprise': 0.054274959256872535, 'neutral': 4.3707069009542465}, 'dominant_emotion': 'angry'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  3.67it/s]


[{'gender': {'Woman': 0.2499803202226758, 'Man': 99.75002408027649}, 'dominant_gender': 'Man', 'region': {'x': 257, 'y': 179, 'w': 143, 'h': 143}, 'emotion': {'angry': 22.287511825561523, 'disgust': 0.0007276627457031282, 'fear': 2.527986466884613, 'happy': 0.048308202531188726, 'sad': 28.12347710132599, 'surprise': 0.0004470631665753899, 'neutral': 47.011542320251465}, 'dominant_emotion': 'neutral'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  3.28it/s]

[{'gender': {'Woman': 1.9903475418686867, 'Man': 98.00965189933777}, 'dominant_gender': 'Man', 'region': {'x': 298, 'y': 186, 'w': 160, 'h': 160}, 'emotion': {'angry': 34.91470993581892, 'disgust': 0.0009584817911398884, 'fear': 16.030664321104965, 'happy': 0.002268093615126812, 'sad': 43.38397760582237, 'surprise': 0.0027688131350133744, 'neutral': 5.664636637015744}, 'dominant_emotion': 'sad'}]
